In [6]:
pip install requests_html

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install hrequests

Defaulting to user installation because normal site-packages is not writeable
  Using cached hrequests-0.8.2-py3-none-any.whl.metadata (40 kB)
  Using cached aioprocessing-2.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached async-class-0.5.0.tar.gz (5.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached faust_cchardet-2.1.19-cp312-cp312-win_amd64.whl.metadata (8.8 kB)
  Using cached gevent-24.2.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached geventhttpclient-2.3.1-cp312-cp312-win_amd64.whl.metadata (10.0 kB)
  Using cached orjson-3.10.5-cp312-none-win_amd64.whl.metadata (50 kB)
  Using cached zope.event-5.0-py3-none-any.whl.metadata (4.4 kB)
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/78.3 k

In [1]:
import requests
import random 
import hrequests
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
# import nest_asyncio
import pyppeteer
import time
from urllib.parse import urljoin
import urllib
import pandas as pd
import time 


import asyncio
if asyncio.get_event_loop().is_running(): # Only patch if needed (i.e. running in Notebook, Spyder, etc)
    import nest_asyncio
    nest_asyncio.apply()

USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0 Safari/537.36',
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36 Edg/103.0.0.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0"
]

headers = {
    'User-Agent': random.choice(USER_AGENTS),
}
asession = AsyncHTMLSession()

WARNING: Please run pip install hrequests[all] for automated browsing support.

In [15]:
product_links = []
# current_price_list = []
# rating_list = []
# trending_list = []
# original_price_list = []
# discount_percentage_list = []
results = []
min_pages = 1
max_pages = 2

for x in range (min_pages, max_pages+1): 
    url = "https://www.zalora.sg/c/men/sports/shoes/c-471/scat-557?page="
    response = await asession.get(url+str(x), headers=headers)
    await response.html.arender(sleep=1, timeout=20)
    soup = BeautifulSoup(response.html.html, 'html.parser')
    # print (response.html.html)

    product_list = soup.find_all('a', class_ = "relative")
    
    for item in product_list: 
        link = item.get("href")
        if link[:5] == "https" or link[:4] == "http": 
        # print (type(link))
        # https://www.zalora.sg/p/camper-camper-men-basket-peu-cami-black-k100249-012-black-3466160 
            product_links.append(link)

    # # Get Current Prices from Main Pages - Completed
        response1 = item.find("span", {'class' : "font-bold text-base"})
        if response1: 
            special_price = response1.get_text()
        else: 
            response1 = item.find("div", {"data-test-id" : "specialPrice"})
            if response1 and response1.find("span") != None:
                special_price = response1.find("span")
            else: 
                if response1 and response1.find("span") == None: 
                    special_price = response1.get_text()
                else: 
                    response1 = item.find("div", {"class" : "flex flex-wrap gap-1 font-medium"})
                    if response1: 
                        special_price = response1.find("div").get_text()
        # current_price_list.append(special_price)
    
    # Get Original Prices (if have) Otherwise display Current Price - All Done
        response2 = item.find("div", {"class" : "text-grey-60 line-through"})
        if response2: 
            original_price = response2.get_text()
        elif response2 is None: 
            # response2 = item.find("div", {"data-test-id" : "originalPrice"})
            original_price = special_price
        # original_price_list.append(original_price)
            

    # Get Ratings of Products - All complete, need to perform EDA to remove unnecessary rows
        rating = item.find("span", {"data-test-id" : "rating"})
        if rating is not None: 
            rating = rating.get_text()
            # rating_list.append(rating)
        else: 
            rating = 'NIL'
            # rating_list.append(rating)
            
    # Get Trending Tag of Products -- Most Null Cases occurred. 
        trending_text = ''
        trending = item.find("div", {"class" : "flex items-center gap-x-[1px] whitespace-nowrap rounded p-0.5 text-xxxs font-semibold desktop:text-xxs bg-[#E5F2FF]"})
        if trending is not None: 
            trending_text = 'Trending'
            # trending_list.append(trending_text)
        else: 
            trending_text = 'NA'
            # trending_list.append(trending_text)
            
    # Get Percentage Discount - All Complete (Need to perform some EDA only)
        discount_percentage = item.find("div", {"data-test-id" : "discountPercentage"})
        if discount_percentage: 
            discounts = discount_percentage.text
            # print (discounts)
            # discount_percentage_list.append(discounts)
        else: 
            discounts = '0%'
            # discount_percentage_list.append('NA')
    
        product_information = {
            'Current Price' : special_price,
            'Original Price' : original_price,
            'Discount Percentage' : discounts,
            'Ratings' : rating,
            'Trending' : trending_text
        }
        results.append(product_information)

# print (discount_percentage_list)   
# print (len(discount_percentage_list))
# print (price_list)
# print (rating_list)
# print (trending_list)
# print (original_price_list)
print ((results))


data = []
for link in product_links: 
    f = hrequests.get(link, headers=headers, timeout=20)
    results = BeautifulSoup(f.content, 'html.parser')

# Get Brand Name
    try: 
        brand_name = results.find("span", {"class" : "mr-2"})
        if brand_name is not None: 
            brand_name = brand_name.get_text("\\n").strip()
    except: 
        brand_name = None
        
# Get Product Name
    try: 
        product_name = results.find("span", {"class" : "mr-2 align-middle text-base text-grey-80"})
        if product_name is not None: 
            product_name = product_name.get_text("\\n").strip()
    except: 
        product_name = None

# Get Variations
    try: 
        variations = results.find("h2", {"class" : "flex overflow-hidden"})
        if variations != None: 
            variations = variations.find("span", {"class" : "truncate text-sm font-normal leading-5 text-neutral-500"}).get_text() 
    except: 
        variations = None
        
# Get Product SKU Number - Currently returns None for all SKU
    try: 
        response4 = results.select("ul", {"class" : "space-y-1"})
        if response4 != None:
            sku = response4.find("li span")
    except: 
        sku = None

# # Get Shoe Size in Country
#     try: 
#         shoe_size_country = results.find_all("button", {"class" : "text-sm outline-none font-medium"}).text
#         if shoe_size_country is not None:
#             shoe_size_country = shoe_size_country.get_text()
#     except: 
#         shoe_size_country = None

    
#     product_details = {
#         "Brand Name" : brand_name,
#         "Product Name" : product_name,
#         "Variations" : variations,        
#         "SKU" : sku,
# # #         # "Shoe Size (country)" : shoe_size_country,
#     }
    
#     data.append(product_details)

# print((data))     

[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s
[D:websockets.client] > %s


[{'Current Price': 'S$\xa0275.00', 'Original Price': 'S$\xa0275.00', 'Discount Percentage': '0%', 'Ratings': 'NIL', 'Trending': 'NA'}, {'Current Price': 'S$\xa0240.00', 'Original Price': 'S$\xa0240.00', 'Discount Percentage': '0%', 'Ratings': '5.0', 'Trending': 'NA'}, {'Current Price': 'S$\xa041.20', 'Original Price': 'S$\xa089.00', 'Discount Percentage': '-54%', 'Ratings': '5.0', 'Trending': 'NA'}, {'Current Price': <span>From S$ 117.90</span>, 'Original Price': 'S$\xa0189.00', 'Discount Percentage': '-38%', 'Ratings': '4.8', 'Trending': 'NA'}, {'Current Price': <span>From S$ 55.90</span>, 'Original Price': 'S$\xa089.00', 'Discount Percentage': '-37%', 'Ratings': '4.9', 'Trending': 'NA'}, {'Current Price': 'S$\xa062.90', 'Original Price': 'S$\xa099.00', 'Discount Percentage': '-36%', 'Ratings': '4.8', 'Trending': 'NA'}, {'Current Price': 'S$\xa0103.95', 'Original Price': 'S$\xa0189.00', 'Discount Percentage': '-45%', 'Ratings': '4.9', 'Trending': 'NA'}, {'Current Price': <span>From S$

In [22]:
df = pd.DataFrame(data, columns = ['Brand Name', 'Product Name'])
df.to_csv("Zalora Scraped Product Details.csv")
